In [25]:
import json
import random
import pandas as pd
from tqdm import tqdm
from web3 import Web3
import time
import numpy as np
import matplotlib.cbook as cbook
import matplotlib.pyplot as plt


In [2]:

'''
These are two functions which will be used to establish/destablish connection between GRID,
based on this we will add/remove connection in GRID)connections e.g. A1-A2, A1-A3, etc.

And also it will be used to transact on the blockchain, which will record the connections and disconnections
'''

def establish_connection(from_,to_):
    #adding connections to GRID_connections set.
    try:
        GRID_connections.add(from_ +'-'+ to_)
        #Interacting with blockchain to transact (enableConnections).
        contract_MappingAddressConnection.functions.enableConnections(from_,to_).transact({'from':accounts_list[0]})
        return "Connection Establihed"
    except :
        return "Connection Already Exists"
        #later try except Exception as error to read from response

def destablish_connection(from_,to_):
    #removing connections from GRID_connections set.

    try:
        GRID_connections.remove(from_ +'-'+ to_)
        #Interacting with blockchain to transact.
        contract_MappingAddressConnection.functions.disableConnections(from_,to_).transact({'from':accounts_list[0]})
        return "Connection Destablished"
    except :
        return "No connection exists to disconnect"

def dataFrame_to_Dict(data):
    #data_dictionary = {}
    data_list = []
    for i,j in data.iteritems():
        dict_j = []
        for z in j:
            dict_j.append(str(z))
        #data_dictionary[str(i)] = dict_j
        data_list.append(dict_j)
    #return data_dictionary
    return data_list

def data_transaction(sender,receiver,iteration,amount):
        print(sender,receiver,iteration,amount)
        try:
            contract_passingArbitraryArguments.functions.passingValues(sender,receiver,iteration,amount).transact({'from':sender})
            return "OKo!"
        except:
            return "No connection"

In [3]:
#max_iteration = [0,1,7,15,31,63,127]
max_iteration_number = 2

In [4]:

#Establising Connections on blockchain
#Connection to Blockchain
web3 = Web3(Web3.HTTPProvider('HTTP://127.0.0.1:7545'))

accounts_list = web3.eth.accounts

#from index 1 to index 4, they will represent each GRID, so mapping them to regions A1, A2, A3, A4 resp.
GRID_accounts = {}
GRID_connections = set()

#Making Dictionary
for i in range(1,len(accounts_list)):
    GRID_accounts['A'+str(i)] = accounts_list[i]

#First CONTRACT Deployment
with open('build/contracts/MappingAddressConnection.json') as f:
  first_migration = json.load(f)

first_migration_contract_address = first_migration['networks']['5777']['address']
first_migration_abi = first_migration['abi']
contract_MappingAddressConnection = web3.eth.contract(address=first_migration_contract_address, abi = first_migration_abi)

#Adding connections
establish_connection(GRID_accounts['A1'],GRID_accounts['A2'])
establish_connection(GRID_accounts['A1'],GRID_accounts['A3'])
establish_connection(GRID_accounts['A2'],GRID_accounts['A1'])
establish_connection(GRID_accounts['A2'],GRID_accounts['A4'])
establish_connection(GRID_accounts['A3'],GRID_accounts['A1'])
establish_connection(GRID_accounts['A3'],GRID_accounts['A4'])
establish_connection(GRID_accounts['A4'],GRID_accounts['A2'])
establish_connection(GRID_accounts['A4'],GRID_accounts['A3'])

A1_A2 = pd.read_csv('../Seperated_Excel_WD/A1_A2.csv', header=None)
A1_A3 = pd.read_csv('../Seperated_Excel_WD/A1_A3.csv', header=None)
A2_A1 = pd.read_csv('../Seperated_Excel_WD/A2_A1.csv', header=None)
A2_A4 = pd.read_csv('../Seperated_Excel_WD/A2_A4.csv', header=None)
A3_A1 = pd.read_csv('../Seperated_Excel_WD/A3_A1.csv', header=None)
A3_A4 = pd.read_csv('../Seperated_Excel_WD/A3_A4.csv', header=None)
A4_A2 = pd.read_csv('../Seperated_Excel_WD/A4_A2.csv', header=None)
A4_A3 = pd.read_csv('../Seperated_Excel_WD/A4_A3.csv', header=None)

#Importing CSV
#interating through data and putting in dictionary
data_A1_A2 = dataFrame_to_Dict(A1_A2)
data_A1_A3 = dataFrame_to_Dict(A1_A3)
data_A2_A1 = dataFrame_to_Dict(A2_A1)
data_A2_A4 = dataFrame_to_Dict(A2_A4)
data_A3_A1 = dataFrame_to_Dict(A3_A1)
data_A3_A4 = dataFrame_to_Dict(A3_A4)
data_A4_A2 = dataFrame_to_Dict(A4_A2)
data_A4_A3 = dataFrame_to_Dict(A4_A3)

#Data connection set:
GRID_Connections = {'A1_A2' : data_A1_A2,
                    'A1_A3' : data_A1_A3,
                    'A2_A1' : data_A2_A1,
                    'A2_A4' : data_A2_A4,
                    'A3_A1' : data_A3_A1,
                    'A3_A4' : data_A3_A4,
                    'A4_A2' : data_A4_A2,
                    'A4_A3' : data_A4_A3
                   }
#Second CONTRACT Deployment
with open('build/contracts/passingArbitraryArguments.json') as f:
  second_migration = json.load(f)

second_migration_contract_address = second_migration['networks']['5777']['address']
second_migration_abi = second_migration['abi']

contract_passingArbitraryArguments = web3.eth.contract(address=second_migration_contract_address, abi = second_migration_abi)

counter = 0
processing = True
max_iteration = 0
while processing and max_iteration != max_iteration_number+1:
    for key in GRID_Connections.keys():
        try:
            sender, receiver = key.split("_")
            payload = GRID_Connections[key][counter]

            print(data_transaction(GRID_accounts[sender],GRID_accounts[receiver],counter,payload))
#                 print(f'''
#                 sender  : {sender}
#                 receiver: {receiver}
#                 counter : {counter}
#                 payload : {payload}
#                 ''')
        except IndexError:
            processing = False
            break
    counter += 1
    max_iteration += 1

destablish_connection(GRID_accounts['A1'],GRID_accounts['A2'])
destablish_connection(GRID_accounts['A1'],GRID_accounts['A3'])
destablish_connection(GRID_accounts['A2'],GRID_accounts['A1'])
destablish_connection(GRID_accounts['A2'],GRID_accounts['A4'])
destablish_connection(GRID_accounts['A3'],GRID_accounts['A1'])
destablish_connection(GRID_accounts['A3'],GRID_accounts['A4'])
destablish_connection(GRID_accounts['A4'],GRID_accounts['A2'])
destablish_connection(GRID_accounts['A4'],GRID_accounts['A3'])

0x81acec1918758C0C8699BcB8D9198d926dbCB3d0 0xb95C58f01dCF90444A6b6d02B7072AD83ecB2152 0 ['0', '0', '1', '1']
OKo!
0x81acec1918758C0C8699BcB8D9198d926dbCB3d0 0xc1cDF74Df03113a8Cfb2Acf31e5A7D3f6BF46ADF 0 ['0', '1']
OKo!
0xb95C58f01dCF90444A6b6d02B7072AD83ecB2152 0x81acec1918758C0C8699BcB8D9198d926dbCB3d0 0 ['0', '1']
OKo!
0xb95C58f01dCF90444A6b6d02B7072AD83ecB2152 0x3bb53eae9172f81490D21cC580117F878Bd05039 0 ['0', '1']
OKo!
0xc1cDF74Df03113a8Cfb2Acf31e5A7D3f6BF46ADF 0x81acec1918758C0C8699BcB8D9198d926dbCB3d0 0 ['0', '1']
OKo!
0xc1cDF74Df03113a8Cfb2Acf31e5A7D3f6BF46ADF 0x3bb53eae9172f81490D21cC580117F878Bd05039 0 ['0', '0', '1', '1']
OKo!
0x3bb53eae9172f81490D21cC580117F878Bd05039 0xb95C58f01dCF90444A6b6d02B7072AD83ecB2152 0 ['0', '0', '1', '1']
OKo!
0x3bb53eae9172f81490D21cC580117F878Bd05039 0xc1cDF74Df03113a8Cfb2Acf31e5A7D3f6BF46ADF 0 ['0', '0', '1', '1']
OKo!
0x81acec1918758C0C8699BcB8D9198d926dbCB3d0 0xb95C58f01dCF90444A6b6d02B7072AD83ecB2152 1 ['-0.0110730297608', '-0.0110730297608',

'Connection Destablished'

In [20]:
# def time_calculation(from_,to_,iteration_number):
#     start_time = time.time()
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[from_],"to": accounts_list[to_],"iteration":iteration_number})
#     event_filter.get_all_entries()
#     end_time = time.time()
#     return (end_time-start_time)
#     #return event_filter.get_all_entries()

# average_iteration_time_of_rounds = []
# y_max_list = []
# y_min_list = []

# #time to retrieve data (filtering:)
# for i in range(max_iteration):
#     print(i)
    
#     iter_1 = time_calculation(1,2,i)
#     print(iter_1)
#     average_of_all_grid_iteration_time.append(iter_1)
#     iter_2 = time_calculation(1,3,i)
#     print(iter_2)
#     average_of_all_grid_iteration_time.append(iter_2)
#     iter_3 = time_calculation(2,1,i)
#     print(iter_3)
#     average_of_all_grid_iteration_time.append(iter_3)
#     iter_4 = time_calculation(2,4,i)
#     print(iter_4)
#     average_of_all_grid_iteration_time.append(iter_4)
#     iter_5 = time_calculation(3,1,i)
#     print(iter_5)
#     average_of_all_grid_iteration_time.append(iter_5)
#     iter_6 = time_calculation(3,4,i)
#     print(iter_6)
#     average_of_all_grid_iteration_time.append(iter_6)
#     iter_7 = time_calculation(4,2,i)
#     print(iter_7)
#     average_of_all_grid_iteration_time.append(iter_7)
#     iter_8 = time_calculation(4,3,i)
#     print(iter_8)
#     average_of_all_grid_iteration_time.append(iter_8)
#     print()
    
#     average_time_of_iteration = (iter_1+iter_2+iter_3+iter_4+iter_5+iter_6+iter_7+iter_8)/8
#     y_max = max([iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8])
#     y_max_list.append(y_max)
#     y_min = min([iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8])
#     y_min_list.append(y_min)
    
#     print(average_time_of_iteration)
#     average_iteration_time_of_rounds.append(average_time_of_iteration)

# print(y_max_list)
# print(average_iteration_time_of_rounds)
# print(y_min_list)

#     #A1-A2
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[1],"to": accounts_list[2],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()
#     #A1-A3
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[1],"to": accounts_list[3],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()

#     #A2-A1
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[2],"to": accounts_list[1],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()

#     #A2-A4
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[2],"to": accounts_list[4],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()

#     #A3-A1
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[3],"to": accounts_list[1],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()

#     #A3-A4
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[3],"to": accounts_list[4],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()    

#     #A4-A2
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[4],"to": accounts_list[2],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()

#     #A4-A3
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[4],"to": accounts_list[3],"iteration":i})
#     print(event_filter.get_all_entries())
#     print()

0
0.04445481300354004
0.04836893081665039
0.0426332950592041
0.08164095878601074
0.03918600082397461
0.03879070281982422
0.03818988800048828
0.03838515281677246

0.046456217765808105
1
0.040023088455200195
0.03820300102233887
0.0409550666809082
0.08649325370788574
0.0369420051574707
0.03721880912780762
0.04581189155578613
0.047547101974487305

0.046649277210235596
2
0.0374598503112793
0.034136056900024414
0.04496312141418457
0.037049055099487305
0.035714149475097656
0.03658795356750488
0.044419050216674805
0.04280996322631836

0.03914240002632141
[0.08164095878601074, 0.08649325370788574, 0.04496312141418457]
[0.046456217765808105, 0.046649277210235596, 0.03914240002632141]
[0.03818988800048828, 0.0369420051574707, 0.034136056900024414]


In [83]:
# def time_calculation(from_,to_,iteration_number):
#     start_time = time.time()
#     event_filter = contract_passingArbitraryArguments.events.NewTrade.createFilter(fromBlock=0,toBlock = 'latest',argument_filters={"_from":accounts_list[from_],"to": accounts_list[to_],"iteration":iteration_number})
#     event_filter.get_all_entries()
#     end_time = time.time()
#     return (end_time-start_time)
#     #return event_filter.get_all_entries()

# average_iteration_time_of_rounds = []
# y_max_list = []
# y_min_list = []

# #time to retrieve data (filtering:)
# for i in range(max_iteration):
#     print(i)
    
#     iter_1 = time_calculation(1,2,i)
#     print(iter_1)
#     average_of_all_grid_iteration_time.append(iter_1)
#     iter_2 = time_calculation(1,3,i)
#     print(iter_2)
#     average_of_all_grid_iteration_time.append(iter_2)
#     iter_3 = time_calculation(2,1,i)
#     print(iter_3)
#     average_of_all_grid_iteration_time.append(iter_3)
#     iter_4 = time_calculation(2,4,i)
#     print(iter_4)
#     average_of_all_grid_iteration_time.append(iter_4)
#     iter_5 = time_calculation(3,1,i)
#     print(iter_5)
#     average_of_all_grid_iteration_time.append(iter_5)
#     iter_6 = time_calculation(3,4,i)
#     print(iter_6)
#     average_of_all_grid_iteration_time.append(iter_6)
#     iter_7 = time_calculation(4,2,i)
#     print(iter_7)
#     average_of_all_grid_iteration_time.append(iter_7)
#     iter_8 = time_calculation(4,3,i)
#     print(iter_8)
#     average_of_all_grid_iteration_time.append(iter_8)
#     print()
    
#     average_time_of_iteration = (iter_1+iter_2+iter_3+iter_4+iter_5+iter_6+iter_7+iter_8)/8
#     y_max = max([iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8])
#     y_max_list.append(y_max)
#     y_min = min([iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8])
#     y_min_list.append(y_min)
    
#     print(average_time_of_iteration)
#     average_iteration_time_of_rounds.append(average_time_of_iteration)

# y_max_list = sum(y_max_list)
# average_iteration_time_of_rounds = sum(average_iteration_time_of_rounds)
# y_min_list = sum(y_min_list)

0
0.05377936363220215
0.04127788543701172
0.05501198768615723
0.03717970848083496
0.04923605918884277
0.038446903228759766
0.03821921348571777
0.03960299491882324

0.0440942645072937
1
0.039092063903808594
0.0823061466217041
0.04652810096740723
0.03986668586730957
0.04503798484802246
0.04180121421813965
0.036547183990478516
0.03953218460083008

0.046338945627212524
2
0.03979802131652832
0.08284163475036621
0.04120922088623047
0.03848981857299805
0.036309003829956055
0.036602020263671875
0.03755593299865723
0.034178972244262695

0.04337307810783386
0.22015976905822754
[0.0440942645072937, 0.046338945627212524, 0.04337307810783386]
[0.03717970848083496, 0.036547183990478516, 0.034178972244262695]
